# Exercise 2 – Naive Bayes Multilabel Classification
(10 points)

Implement a Naive Bayes classifier which is able to assign multiple classes to a single document by finalizing the two given classes. The `MultiLabelLearner` class acts like a builder for the `MultiLabelClassifier` instances. That means that the learner gets the number of classes during its creation and the `learnExample` method of the learner is called once for each document of the training set. Internally, the learner should gather all statistics that are necessary for the classifier when processing the training examples.
After the learner saw all training documents, the `createClassifier` method is called which creates an instance of the `MultiLabelClassifier` class and initializes it with the statistics gathered before. 
The classification itself is carried out by the `classify` method which takes an unknown document and assigns it a set of classes learned before.

#### Hints

- Please do not forget to preprocess your documents. What exactly the preprocessing does is up to you.
- The classification should be based on the naive Bayes classification. You may want to reuse code from exercise 1.
- In our datasets, each document has *at least one* class. You may want to take this information into account.
- The evaluation will use micro precision, micro recall and micro F1-measure (also named F1-score).
- The evaluation in the hidden tests has three stages. 
  1. Your solution will get 4 points as soon as it is better than the baselines. The baselines are:
     - For each class, a classifier that always returns this class.
     - A random guesser that returns a random class.
  2. If your solution has an F1-score >= 0.7, you will get 3 more points.
  3. If your solution has an F1-score >= 0.8, you will get 3 more points.
- You can download the [multi-class-train.tsv](https://hobbitdata.informatik.uni-leipzig.de/teaching/SNLP/classification/multi-class-train.tsv) file. It comprises one document per line. The first part comprises the classes (separated with a `", "` string), followed by a tab character (`\t`). The remaining content of the line is the text of the document.

#### Notes

- You are free to use a different IDE to develop your solution. However, you have to copy the solution into this notebook to submit it.
- Do not add additional external libraries.
- Interface
  - You can use _[TAB]_ for autocompletion and _[SHIFT]_+_[TAB]_ for code inspection.
  - Use _Menu_ -> _View_ -> _Toggle Line Numbers_ for debugging.
  - Check _Menu_ -> _Help_ -> _Keyboard Shortcuts_.
- Known issues
  - All global variables will be set to void after an import.
  - Missing spaces arround `%` (Modulo) can cause unexpected errors so please make sure that you have added spaces around every `%` character.
- Finish
  - Save your solution by clicking on the _disk icon_.
  - Make sure that all necessary imports are listed at the beginning of your cell.
  - Run a final check of your solution by
    - click on _restart the kernel, then re-run the whole notebook_ (the fast forward arrow in the tool bar)
    - wait fo the kernel to restart and execute all cells (all executable cells should have numbers in front of them instead of a `[*]`) 
    - Check all executed cells for errors. If an exception is thrown, please check your code. Note that although the error might look cryptic, until now we never encounter that an exception was caused without a valid reason inside of the submitted code. A good way to check the code is to copy the solution into a new class in your favorite IDE and check
      - errors reported by the IDE
      - imports the IDE adds to your code which might be missing in your submission.
  - Finally, choose _Menu_ -> _File_ -> _Close and Halt_.
  - Do not forget to _Submit_ your solution in the _Assignments_ view.

In [3]:
// package NaiveBayesian;

import java.util.ArrayList;
import java.util.Arrays;
import java.util.Collections;
import java.util.HashMap;
import java.util.HashSet;
import java.util.List;
import java.util.Map;
import java.util.Set;

//YOUR CODE HERE

/**
 * Classifier implementing naive Bayes classification for a multilabel
 * classification.
 */
public class MultiLabelClassifier {
	// YOUR CODE HERE
	List<String> classNameList;
	int totalDocCount;
	Map<String, int[]> classFreqMap;
	List<String> vocab1;
	int[] docCountEachClass;
	double[] priorProbs;
	Map<String, Integer> classDocsCount;

	MultiLabelClassifier() {
	};

	public MultiLabelClassifier(List<String> classNameList, int totalDocCount, Map<String, int[]> classFreqMap,
			List<String> vocab1, int[] docCountEachClass, Map<String, Integer> classDocsCount) {
		// TODO Auto-generated constructor stub
		this.classFreqMap = classFreqMap;
		this.classNameList = classNameList;
		this.totalDocCount = totalDocCount;
		this.vocab1 = vocab1;
		this.docCountEachClass = docCountEachClass;
		this.classDocsCount = classDocsCount;
		
		priorProbs = new double[docCountEachClass.length];
		// Now we have the prior probs of each class
		for (int i = 0; i < priorProbs.length; i++) {
			priorProbs[i] = (double) docCountEachClass[i] / totalDocCount;
		}

	}

	String[] stopwords = { "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
			"yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its",
			"itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
			"these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having",
			"do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
			"of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
			"after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
			"further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
			"few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
			"too", "very", "s", "t", "can", "will", "just", "don", "should", "now" };

	List<String> stopWordsList = Arrays.asList(stopwords);

	public Set<String> preprocessing(String text) {
//		System.out.println(text);
		double[][] condProbArray = new double[vocab1.size()][classNameList.size()];

		String[] tokens = text.toLowerCase().replaceAll("[^ a-zA-Z0-9]", "").split(" ");
		double score[] = new double[classNameList.size()];

		for (String token : tokens) {
			// check if the token is not a stop word
			if (!stopWordsList.contains(token)) {
				int indexToken = 0, freqToken = 0;
				boolean flag = false;

				if (vocab1.contains(token)) {
					indexToken = vocab1.indexOf(token);
					flag = true; // that token is in vocab
				}

				double condProb = 0.0;

				int i = 0;
				// calculate the score of each class for this token.
				for (String c : classNameList) {
					// this will hold the total length of each class
					int classSize = classDocsCount.get(c);

					// pull out the array of frequence for each term for that
					// class
					int[] wordList = classFreqMap.get(c);

					double logToken;

					if (flag) {

						condProb = condProbArray[indexToken][classNameList.indexOf(c)];
						if (condProb > 0) {
							// as the condprob is already there, calculate its
							// log
							logToken = Math.log(condProb);
							score[i] += logToken;
							i++;
							continue;
						}
						// if the condProb is not there, calculate
						else {
							// our aim here is to calculate only the freqToken
							// so it can be zero if the token is not in vocab.
							freqToken = wordList[indexToken];
							condProbArray[indexToken][classNameList.indexOf(c)] = (double) (freqToken + 1)
									/ (classSize + vocab1.size());
							logToken = Math.log(condProbArray[indexToken][classNameList.indexOf(c)]);
							score[i] += logToken;
							i++;
							continue;
						}
					}

					// here if token is not in vocab then freq would be zero.

					// use this freq for condProbArray
					else {

						condProb = (double) (freqToken + 1) / (classSize + vocab1.size());
						logToken = Math.log(condProb);
						score[i] += logToken;
						i++;
						continue;
					}

				}

			}

		}
		
		for (int i = 0; i < score.length; i++) {
			score[i] += Math.log(priorProbs[i]);
//			System.out.println("or score: "+score[i]);
		}
		
//		for (int i = 0; i < score.length; i++) {
//			System.out.println(score[i]+" "+classNameList.get(i));
//		}
		
//		Arrays.sort(score);
		List<Double> scoreList = new ArrayList<>();
		
		for(double scoreElement: score){
			scoreList.add(scoreElement);
		}
		Collections.sort(scoreList);
		double maxScore = scoreList.get(scoreList.size()-1);
		Set<String> results = new HashSet();
		for(int k=0;k<score.length;k++){
//			System.out.println(score[k]);
			if(maxScore-score[k]<1.5){
				results.add(classNameList.get(k));
			}
		}
		
		
//		System.out.println("the final result: "+results);
		return results;
	}

	/**
	 * Classifies the given document and returns the class names.
	 */
	public Set<String> classify(String text) {
		Set<String> score = preprocessing(text);
		
		for(int i=0;i<classNameList.size();i++){
//			System.out.println(classNameList.get(i));
		}
		
		/*for(int i =0;i<score.length;i++){
			System.out.println(text+" score of "+ i+ score[i]);
		}
		Set<String> results = new HashSet<>();
		double maxScore = score[(score.length)-1];
		results.add(classNameList.get(score.length-1));
		System.out.println(results+""+maxScore);
		for(int d=0;d<score.length-1;d++){
			if(maxScore-score[d]<0.8){
				results.add(classNameList.get(d));
				System.out.println(results);
			}
		}
		*/
//		System.out.println(text);
		// YOUR CODE HERE
//		results.add("history");
		return score;
	}
}

/**
 * Learner (or Builder) class for a naive Bayes multilabel classifier.
 */
class MultiLabelLearner {
	// YOUR CODE HERE
	List<String> classNameList = new ArrayList<>();
	int totalDocCount = 0; // stores the total number of classes.
	int[] docCountEachClass; // stores the count of each class at index.
	List<String> vocab1 = new ArrayList<>();

	// this map stores the class and all the docs related to the class in one
	// string object
	Map<String, String> mapClassText = new HashMap<>();

	/**
	 * Constructor taking the number of classes the classifier should be able to
	 * distinguish.
	 */
	public MultiLabelLearner(Set<String> classes) {
		// YOUR CODE HERE

		for (String clazz : classes) {
			classNameList.add(clazz);
		}
		

		docCountEachClass = new int[classNameList.size()];
	}

	String[] stopwords = { "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
			"yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its",
			"itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
			"these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having",
			"do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
			"of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
			"after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
			"further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
			"few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
			"too", "very", "s", "t", "can", "will", "just", "don", "should", "now" };

	List<String> stopWordsList = Arrays.asList(stopwords);

	// This array belongs to each class containing for each class, its term
	// count array.
	Map<String, int[]> classFreqMap = new HashMap<>();
	// This array belongs to mapping of count of term with vocab
	int[] tokenCountEachVocab;

	/**
	 * The method used to learn the training examples. It takes the names of the
	 * classes as well as the text of the training document.
	 */
	public void learnExample(Set<String> classes, String text) {
		// YOUR CODE HERE
		totalDocCount++;

		// here we have calculated the array with no. of documents belonging to
		// a class.
//		if(classes.size()==1){
		for (String clazz : classes) {
			int indexClass = classNameList.indexOf(clazz);
			docCountEachClass[indexClass] += 1;

			// Performance measure test with the preprocessed textual content
			String text_processed = text.toLowerCase().replaceAll("[^ a-zA-Z0-9]", "");

			// we preprocess a text for that class. This will concatenate the
			// texts for one class and also prepare the vocab.
			addVocab(text_processed);

			preprocessing(clazz, text_processed);
		}
	}

	void addVocab(String text) {
		String tokens[] = text.split(" ");
		for (String token : tokens) {
			if (!stopWordsList.contains(token)) {
				if (!vocab1.contains(token)) {
					vocab1.add(token);
				}
			}
		}
	}

	void preprocessing(String clazz, String text) {
		// This method, will assign a concatenated string to class.
		String tempVect = new String();
		if (mapClassText.containsKey(clazz)) {
			tempVect = mapClassText.get(clazz);
		}

		mapClassText.put(clazz, tempVect.concat(" "+text));

	}

	Map<String, Integer> classDocsCount = new HashMap<>();

	int[] prepareArray(String[] tokens) {
		tokenCountEachVocab = new int[vocab1.size()];

		for (String token : tokens) {
			if (!stopWordsList.contains(token) && !token.equals("")) {
				int indexTokenVocab = vocab1.indexOf(token);
				tokenCountEachVocab[indexTokenVocab] += 1;
			}
		}

		return tokenCountEachVocab;
	}

	/**
	 * Creates a MultiLabelClassifier instance based on the statistics gathered
	 * from the training example.
	 */
	public MultiLabelClassifier createClassifier() {
		// YOUR CODE HERE
		for (String clazz : classNameList) {
			String textClass = mapClassText.get(clazz);
			String[] tokens = textClass.trim().split(" ");
			int[] tokenSpace = prepareArray(tokens);
			int sum=0;
			for(int t:tokenSpace){
				sum+=t;
			}
			// This will store for each class, no. of tokens belonging to it.
			classDocsCount.put(clazz, sum);
			classFreqMap.put(clazz, tokenSpace);
		}

		MultiLabelClassifier classifier = new MultiLabelClassifier(classNameList, totalDocCount, classFreqMap, vocab1,
				docCountEachClass, classDocsCount);

		return classifier;
	}
}
// This line should make sure that compile errors are directly identified when
// executing this cell
// (the line itself does not produce any meaningful result)
new MultiLabelLearner(new HashSet<>(Arrays.asList("good","bad")));
System.out.println("compiled");

compiled


# Evaluation

- Run the following cell to test your implementation.
- You can ignore the cells afterwards.

In [4]:
%maven org.junit.jupiter:junit-jupiter-api:5.3.1
%maven commons-io:commons-io:2.6
import org.junit.jupiter.api.Assertions;
import org.opentest4j.AssertionFailedError;
import java.util.stream.Collectors;
import java.util.Map.Entry;
import org.apache.commons.io.FileUtils;
import java.io.File;
import java.io.IOException;

/**
 * Simple structure to store the classes and the text of a document.
 */
public class ClassifiedDocument {
    public final Set<String> classes;
    public final String text;
    public ClassifiedDocument(Set<String> classes, String text) {
        this.classes = classes;
        this.text = text;
    }
}
/**
 * Simple method for reading classification examples from a file as a list of (classes, text) pairs.
 */
public static List<ClassifiedDocument> readClassData(String filename) throws IOException {
    return FileUtils.readLines(new File(filename), "utf-8").stream().map(s -> s.split("\t"))
            .filter(s -> s.length > 1)
            .map(s -> new ClassifiedDocument(new HashSet<>(Arrays.asList(s[0].split(", "))), s[1]))
            .collect(Collectors.toList());
}

/**
 * Method for cecking the given classifier. The method expects training and evaluation data.
 * The data should have a String array for each document in which the first cell of the
 * array contains the class while the second cell contains the text of the document. During
 * the check, some statistics like the F1-scores of different baseline classifiers are
 * printed. Finally, the calculated F1-score is returned.
 *
 * @param trainingCorpus the data that is used for training the classifier. 
 * @param evaluationCorpus the data that is used for evaluating the classifier. 
 * @param minAccuracy minimum accuracy the classifier should achieve.
 * @return the F1-score achieved by the classifier
 */
public static double checkClassifier(List<ClassifiedDocument> trainingCorpus,
        List<ClassifiedDocument> evaluationCorpus, double minF1Score) {
    double f1 = 0;
    try {
        System.out.print("Training corpus size: ");
        System.out.println(trainingCorpus.size());
        System.out.print("Eval. corpus size   : ");
        System.out.println(evaluationCorpus.size());
        // Determine the classes
        Set<String> classes = Arrays.asList(trainingCorpus, evaluationCorpus).stream().flatMap(l -> l.stream())
                .map(d -> d.classes).flatMap(c -> c.stream()).distinct().collect(Collectors.toSet());
        // Determine the number of instances per class in the evaluation set
        Map<String, Long> evalClassCounts = evaluationCorpus.stream().map(d -> d.classes).flatMap(c -> c.stream())
                .collect(Collectors.groupingBy(c -> c, Collectors.counting()));
        for (String clazz : classes) {
            if (!evalClassCounts.containsKey(clazz)) {
                evalClassCounts.put(clazz, 0L);
            }
        }
        long expectedClassSum = evalClassCounts.entrySet().stream().mapToLong(e -> e.getValue()).sum();

        // Determine the expected accuracies of the baselines
        Map<String, double[]> f1ForClassGuessers = new HashMap<>();
        for (Entry<String, Long> e : evalClassCounts.entrySet()) {
            f1ForClassGuessers.put(e.getKey(), calcStats(e.getValue().intValue(),
                    evaluationCorpus.size() - e.getValue().intValue(), (int) (expectedClassSum - e.getValue())));
        }

        // Train the classifier
        long time1 = System.currentTimeMillis();
        MultiLabelLearner learner = new MultiLabelLearner(classes);
        for (ClassifiedDocument trainingExample : trainingCorpus) {
            learner.learnExample(trainingExample.classes, trainingExample.text);
        }
        MultiLabelClassifier classifier = learner.createClassifier();
        time1 = System.currentTimeMillis() - time1;
        System.out.println("Training took       : " + time1 + "ms");

        // Classify the evaluation corpus
        long time2 = System.currentTimeMillis();
        Map<String, int[]> classCounts = new HashMap<>();
        final int TP = 0, FP = 1, FN = 2, TN = 3;
        for (String clazz : classes) {
            classCounts.put(clazz, new int[4]);
        }
        int id = 0;
        Set<String> result;
        List<String[]> errorDetails = new ArrayList<>();
        boolean added;
        for (ClassifiedDocument evalExample : evaluationCorpus) {
            added = false;
            result = classifier.classify(evalExample.text);
            String resultAsString = result.toString();
            for (String clazz : classes) {
                if (evalExample.classes.contains(clazz)) {
                    if (result.contains(clazz)) {
                        ++classCounts.get(clazz)[TP];
                    } else {
                        ++classCounts.get(clazz)[FN];
                        if (!added) {
                            errorDetails.add(new String[] { Integer.toString(id), evalExample.classes.toString(),
                                    resultAsString });
                        }
                    }
                } else {
                    if (result.contains(clazz)) {
                        ++classCounts.get(clazz)[FP];
                        if (!added) {
                            errorDetails.add(new String[] { Integer.toString(id), evalExample.classes.toString(),
                                    resultAsString });
                        }
                    } else {
                        ++classCounts.get(clazz)[TN];
                    }
                }
            }
            result.removeAll(evalExample.classes);
            if ((result.size() > 0) && (!added)) {
                errorDetails.add(
                        new String[] { Integer.toString(id), evalExample.classes.toString(), result.toString() });
            }
            ++id;
        }
        time2 = System.currentTimeMillis() - time2;
        System.out.println("Classification took : " + time2 + "ms");
        int counts[] = new int[4];
        for (Entry<String, int[]> stats : classCounts.entrySet()) {
            counts[0] += stats.getValue()[0];
            counts[1] += stats.getValue()[1];
            counts[2] += stats.getValue()[2];
            counts[3] += stats.getValue()[3];
        }
        double solutionPerformance[] = calcStats(counts[TP], counts[FP], counts[FN]);

        System.out.println("classifiers           precision    recall  f1-score");
        for (Entry<String, double[]> baseResult : f1ForClassGuessers.entrySet()) {
            System.out.println(String.format("Always %-13s:   %-7.5f   %-7.5f   %-7.5f", baseResult.getKey(),
                    baseResult.getValue()[0], baseResult.getValue()[1], baseResult.getValue()[2]));
        }
        System.out.println(
                String.format("Your solution       :   %-7.5f   %-7.5f   %-7.5f (%d tp, %d tn, %d fp, %d fn)",
                        solutionPerformance[0], solutionPerformance[1], solutionPerformance[2], counts[TP],
                        counts[TN], counts[FP], counts[FN]));
        f1 = solutionPerformance[2];
        if (errorDetails.size() > 0) {
            System.out.println("  Wrong classifications are:");
            for (int i = 0; i < Math.min(errorDetails.size(), 20); ++i) {
                System.out.print("    id=");
                System.out.print(errorDetails.get(i)[0]);
                System.out.print(" expected=");
                System.out.print(errorDetails.get(i)[1]);
                System.out.print(" result=");
                System.out.println(errorDetails.get(i)[2]);
            }
            if (errorDetails.size() > 20) {
                System.out.println("    ...");
            }
        }

        // Make sure that the students solution is better than all baselines
        for (Entry<String, double[]> baseResult : f1ForClassGuessers.entrySet()) {
            if (baseResult.getValue()[2] >= solutionPerformance[2]) {
                StringBuilder builder = new StringBuilder();
                builder.append("Your solution is not better than a classifier that always chooses the \"");
                builder.append(baseResult.getKey());
                builder.append("\" class.");
                Assertions.fail(builder.toString());
            }
        }
        if ((minF1Score > 0) && (minF1Score > solutionPerformance[2])) {
            Assertions.fail("Your solution did not reach the expected F1-score of " + minF1Score);
        }
        System.out.println("Test successfully completed.");
    } catch (AssertionFailedError e) {
        throw e;
    } catch (Throwable e) {
        System.err.println("Your solution caused an unexpected error:");
        throw e;
    }
    return f1;
}
/**
 * Simple method for calculating micro precision, recall and F1-measure.
 */
public static double[] calcStats(int tp, int fp, int fn) {
    double precision = tp / (double) (tp + fp);
    double recall = tp / (double) (tp + fn);
    return new double[] { precision, recall, (2 * precision * recall) / (precision + recall) };
}

System.out.println("---------- Simple example corpus ----------");
List<ClassifiedDocument> exampleCorpusTrain = Arrays.asList(
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("chess")),
                "white king, black rook, black queen, white pawn, black knight, white bishop."),
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("history")),
                "knight person granted honorary title knighthood"),
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("history")),
                "knight order eligibility, knighthood, head of state, king, prelate, middle ages."),
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("chess", "game")),
                "Defense knight king pawn opening game opponent."),
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("game")),
                "Game. player opponent victory. draw."));
List<ClassifiedDocument> exampleCorpusTest = Arrays.asList(
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("history")), "Knighthood Middle Ages."),
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("game", "chess")),
                "player black knight opponent pawn queen checkmate game draw victory."),
        // document with unknown words
        new ClassifiedDocument(new HashSet<String>(Arrays.asList("game")), "player opponent opening"));
double f1Measure = checkClassifier(exampleCorpusTrain, exampleCorpusTest, 0);

System.out.println();
System.out.println("---------- Larger example corpus ----------");
List<ClassifiedDocument> classificationData = readClassData("/srv/distribution/multi-class-train.tsv");
f1Measure = checkClassifier(classificationData.subList(0, 600), classificationData.subList(600, classificationData.size()),
        0);

---------- Simple example corpus ----------
Training corpus size: 5
Eval. corpus size   : 3
Training took       : 1ms
Classification took : 0ms
classifiers           precision    recall  f1-score
Always game         :   0.66667   0.50000   0.57143
Always chess        :   0.33333   0.25000   0.28571
Always history      :   0.33333   0.25000   0.28571
Your solution       :   1.00000   0.75000   0.85714 (3 tp, 5 tn, 0 fp, 1 fn)
  Wrong classifications are:
    id=1 expected=[game, chess] result=[game]
Test successfully completed.

---------- Larger example corpus ----------
Training corpus size: 600
Eval. corpus size   : 195
Training took       : 2975ms
Classification took : 625ms
classifiers           precision    recall  f1-score
Always money-fx     :   0.43590   0.29720   0.35343
Always nat-gas      :   0.03077   0.02098   0.02495
Always interest     :   0.10256   0.06993   0.08316
Always corn         :   0.06667   0.04545   0.05405
Always ship         :   0.16923   0.11538   0.13721
A

In [3]:
// Ignore this cell

In [4]:
// Ignore this cell

In [5]:
// Ignore this cell